In [1]:
import tensorflow as tf
import pickle
import numpy as np
import timeit
import load_cifar_template as lc
from tensorflow.contrib.layers import flatten
import matplotlib.pyplot as plt
import time

from tqdm import tqdm
import scipy.io as spio
import pickle
import os
import cv2
import timeit
from skimage.transform import rescale, resize, downscale_local_mean

In [10]:
#fetch the data

def fetch_data(directory, label): 
    item_list = []
    step = 0
    for file in tqdm(os.listdir(directory)):
        full_img_str = directory + "/" + file
        #print(full_img_str)

        mat = spio.loadmat(full_img_str, squeeze_me=True)
        data = np.abs(mat["data_store"])
        smaller_data = data[100:400]
        
        image_resized = resize(smaller_data, (100, 320, 4),
                       anti_aliasing=True)
        
        sub_list = []
        for i in range(8):
            chunk = image_resized[:,40*i:40*i+40]
            sub_list.append(chunk)
        
        #
        #print(data.shape)
        #print(data)
        ###append the img and label to the list###
        #sub_list = [image_resized, label]
        #print(sub_list)
        item_list.append([sub_list, label])
        
    return item_list
    
bike1 = fetch_data("Desktop/ee596prepro/2019_04_09_bms1000/data", [1, 0, 0])
bike2 = fetch_data("Desktop/ee596prepro/2019_04_09_bms1001/data", [1, 0, 0])
bike3 = fetch_data("Desktop/ee596prepro/2019_04_09_bms1002/data", [1, 0, 0])

car1 = fetch_data("Desktop/ee596prepro/2019_04_09_cms1000/data", [0, 1, 0])
car2 = fetch_data("Desktop/ee596prepro/2019_04_09_cms1001/data", [0, 1, 0])
car3 = fetch_data("Desktop/ee596prepro/2019_04_09_cms1002/data", [0, 1, 0])

ped1 = fetch_data("Desktop/ee596prepro/2019_04_09_pms1000/data", [0, 0, 1])
ped2 = fetch_data("Desktop/ee596prepro/2019_04_09_pms1001/data", [0, 0, 1])
ped3 = fetch_data("Desktop/ee596prepro/2019_04_09_pms2000/data", [0, 0, 1])



#directory = "Desktop/ee596prepro/2019_04_09_bms1000/data"

100%|████████████████████████████████████████████████████████████████████████████████| 333/333 [00:21<00:00, 15.54it/s]


In [11]:
def mini_batch(features,labels,mini_batch_size):
    """
    Args:
        features: features for one batch
        labels: labels for one batch
        mini_batch_size: the mini-batch size you want to use.
    Hint: Use "yield" to generate mini-batch features and labels
    """
    #split the data into batches
    amount_of_data = len(features)
    number_of_bunches = amount_of_data/mini_batch_size
    
    bunches_features = []
    bunches_labels = []
    
    #loop over breaking the data into batches
    for i in range(int(number_of_bunches)):
        current_range = i * mini_batch_size
        f_b = features[current_range:current_range+mini_batch_size]
        l_b = labels[current_range:current_range+mini_batch_size]
        
        bunches_features.append(f_b)
        bunches_labels.append(l_b)
    
    #return the mini-batched data
    return bunches_features, bunches_labels

#full_list = bike1 + bike2 + bike3 + car1 + car2 + car3 + ped1 + ped2 + ped3
full_list = bike1 + car1 + ped1 + bike2 + car2 + ped2 + bike3 + car3 + ped3
bike1 = None
car1 = None
ped1 = None

#test_list = bike3 + car3 + ped3

np.random.shuffle(full_list)
print(np.asarray(full_list).shape)
#print(np.asarray(full_list)[0][0].shape)
#np.random.shuffle(test_list)

training_set = full_list[0:2700]
valid_set = full_list[2700:2850]
test_set = full_list[2850:2997]
full_list = None

train_set_data = np.zeros((len(training_set), 8, 100, 40, 4))
train_set_labels = np.zeros((len(training_set), 3))
valid_set_data = np.zeros((len(valid_set), 8, 100, 40, 4))
valid_set_labels = np.zeros((len(valid_set), 3))
test_set_data = np.zeros((len(test_set), 8, 100, 40, 4))
test_set_labels = np.zeros((len(test_set), 3))
#print(np.asarray(training_set).shape)
#print(training_set[0][0][0])
#print(training_set[0][:][1])
#print(training_set[0][:][0].shape)
#print(training_set[0][:][0])
#split into training, valid, and testing
for i in range(len(training_set)):
    train_set_data[i] = training_set[i][0]#np.reshape(training_set[i][0], (100, 960, 4))
    train_set_labels[i] = training_set[i][1]
    
for i in range(len(valid_set)):
    valid_set_data[i] = valid_set[i][0]#np.reshape(valid_set[i][0], (100, 960, 4))
    valid_set_labels[i] = valid_set[i][1]

for i in range(len(test_set)):
    test_set_data[i] = test_set[i][0]#np.reshape(test_set[i][0], (100, 960, 4))
    test_set_labels[i] = test_set[i][1]

#print(np.asarray(train_set_data).shape)
#print(train_set_data.shape)


#train_set_data, train_set_labels = mini_batch(train_set_data,train_set_labels,5)
#valid_set_data, valid_set_labels = mini_batch(valid_set_data,valid_set_labels,5)
#test_set_data, test_set_labels = mini_batch(test_set_data,test_set_labels,5)

training_set = None
valid_set = None
test_set = None


#test_list = None

(2997, 2)


In [12]:
print(test_set_data[0].shape)
print(train_set_labels)

(8, 100, 40, 4)
[[0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]


In [14]:
#from keras.models import Sequential
#from keras.layers import Dense
#from keras.layers import LSTM
#import keras.models
#import keras.layers
from tensorflow.keras import metrics
#X = seq.reshape(len(seq), 1, 1)
#y = seq.reshape(len(seq), 1)
seq_imgs = 25
print(np.asarray(train_set_data).shape)
#train_set_data = np.reshape(train_set_data, (980*25, 227, 227, 3))
#print(train_set_labels.shape)


model = tf.keras.models.Sequential()
model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2D(30, (5, 5)),
                              input_shape=(8, 100, 40, 4)))
model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPool2D(2, (2, 2))))
model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten()))
model.add(tf.keras.layers.LSTM(50))
model.add(tf.keras.layers.Dense(3))

adam_opt = tf.keras.optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='categorical_crossentropy', optimizer=adam_opt,
              metrics=[metrics.mae, metrics.categorical_accuracy])

model.fit(train_set_data, train_set_labels, epochs=10, batch_size=5, verbose = 2, validation_data = (valid_set_data, valid_set_labels))



(2700, 8, 100, 40, 4)
Train on 2700 samples, validate on 150 samples
Epoch 1/10
 - 28s - loss: 4.1519 - mean_absolute_error: 0.3639 - categorical_accuracy: 0.3293 - val_loss: 1.1390 - val_mean_absolute_error: 0.3657 - val_categorical_accuracy: 0.3400
Epoch 2/10
 - 26s - loss: 0.9686 - mean_absolute_error: 0.3619 - categorical_accuracy: 0.3559 - val_loss: 0.8827 - val_mean_absolute_error: 0.3563 - val_categorical_accuracy: 0.4533
Epoch 3/10
 - 26s - loss: 1.6223 - mean_absolute_error: 0.3537 - categorical_accuracy: 0.4707 - val_loss: 2.2593 - val_mean_absolute_error: 0.3480 - val_categorical_accuracy: 0.5733
Epoch 4/10
 - 26s - loss: 2.0348 - mean_absolute_error: 0.3470 - categorical_accuracy: 0.5789 - val_loss: 1.9959 - val_mean_absolute_error: 0.3465 - val_categorical_accuracy: 0.5867
Epoch 5/10
 - 27s - loss: 1.7445 - mean_absolute_error: 0.3445 - categorical_accuracy: 0.6122 - val_loss: 2.3259 - val_mean_absolute_error: 0.3412 - val_categorical_accuracy: 0.6067
Epoch 6/10
 - 27s - l

In [15]:
acc = 0

test_set_data = np.asarray(test_set_data)
test_set_labels = np.asarray(test_set_labels)

#fetch batch
#batch_x = test_set_data[k]
#batch_y = test_set_labels[k]
#run optimization
prediction = model.predict(test_set_data, verbose=0)
print(prediction)
for k in range(len(test_set_data)):
    if (np.argmax(prediction[k]) == np.argmax(test_set_labels[k])):
        acc += 1
    else:
        acc += 0
    



acc = acc/len(test_set_data)
print("Test Accuracy= {:.3f}".format(acc))

[[-0.1635845  -0.11875401  0.11665931]
 [-0.12548935 -0.3949679   0.4082563 ]
 [-0.18063098 -0.55348724  0.6013963 ]
 [-0.375394   -0.70537776  0.64938474]
 [-0.12409621 -0.18158218  0.16343269]
 [-0.10267272 -0.15440369  0.22922446]
 [-0.13710463 -0.43713263  0.54479146]
 [-0.24677995 -0.41844806  0.34390646]
 [-0.11664302 -0.1526371   0.26628584]
 [-0.19243817 -0.3535282   0.3481047 ]
 [-0.1899894  -0.37402797  0.40710866]
 [-0.32324946 -0.46262076  0.298719  ]
 [-0.18860681 -0.14852068  0.19166689]
 [-0.18470089 -0.27865     0.27930784]
 [-0.10578776 -0.18295008  0.20010628]
 [-0.12903257 -0.13980007  0.15753324]
 [-0.16592914 -0.191387    0.22905485]
 [-0.10736211 -0.09301118  0.13105856]
 [-0.10737053 -0.16726875  0.18450965]
 [-0.15790533 -0.400402    0.41161984]
 [-0.07126506 -0.13604414  0.22448261]
 [-0.13170786 -0.11084684  0.17201038]
 [-0.10183259 -0.18035975  0.280983  ]
 [-0.28784567 -0.51825756  0.3494769 ]
 [-0.1942939  -0.38144413  0.429895  ]
 [-0.12360065 -0.12778035